# Generating **Completions** with **zyx**

The primary module of zyx, is the universal <code>.completion()</code> function. This module is an extensive wrapper around the <code>litellm .completion()</code> function, as well as the Instructor library. </br>

The <code>.completion()</code> function is capable of

- **Generations with any LiteLLM compatible model**
    - Ollama, OpenAI, Anthropic, Groq, Mistral, and more!
- **Direct Instructor Pydantc structured outputs**
- **Tool calling & execution support. (Get a tool interpretation with one function)**
    - zyx provides a few prebuilt tools out of the box
    - Can take in a list of **Python functions**, **OpenAI dictionaries**, or **Pydantic models** as tools!
    - Automatic tool execution if a tool is called through the <code>run_tools</code> parameter
- **Streaming**
- **New** Vision support 
    - Pass in a list of urls
    - Currently uses multi shot prompting if a response model or tools were also passed.
-  **New** Prompt optimization 
    - Creates or optimizes a task tuned system prompt using either the *COSTAR* or *TIDD-EC* frameworks automatically.

## **Standard Completion**

Lets start by generated a simple LLM completion.

In [1]:
# Simplest Way to Generate
# Defaults to "gpt-4o-mini" if no model is provided
from zyx import completion

response = completion("Hi, how are you?")

# Returns a standard OpenAI style response object
print(response.choices[0].message.content)

/home/hammad/miniconda3/envs/zyx/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


Hello! I'm just a program, but I'm here and ready to help you. How can I assist you today?

The completion function is able to take in messages as a <code>string</code>, or a <code>list of OpenAI formatted messages</code>. </br>

The OpenAI message format is as follows:

```python
[
    {"role": "system", "content": "You are a helpful assistant"}, # System Prompt
    {"role": "user", "content": "Hi, how are you?"} # User Message
    # {"role": "assistant", "content": "I am fine, thank you!"} # Assistant Message
]
```

In [2]:
from zyx import completion

# Lets pass in a system prompt to change the behavior of the assistant
response = completion(
    messages = [
        {"role": "system", "content": "You only respond in spanish"},
        {"role": "user", "content": "Hi how are you?"}
    ],
    model = "ollama/llama3.1", # Any LiteLLM compatible model will work
    max_tokens = 100,
    temperature = 0.5,
)

# Lets print the full response object
print(response)

ModelResponse(
    id='chatcmpl-7894f781-84d4-4a6c-88b7-d0d20ee3f2da',
    choices=[
        Choices(
            finish_reason='stop',
            index=0,
            message=Message(
                content='Estoy bien, gracias. ¿Y tú?',
                role='assistant',
                tool_calls=None,
                function_call=None
            )
        )
    ],
    created=1725776260,
    model='ollama/llama3.1',
    object='chat.completion',
    system_fingerprint=None,
    usage=Usage(completion_tokens=11, prompt_tokens=28, total_tokens=39)
)

## **Generating Structured Outputs with Pydantic & Instructor**

<samp>zyx</samp> builds the <code>instructor</code> wrapper on top of the <code>litellm</code> completion wrapper, so you're able to use the functionality of instructor straight from the same function. </br>

### **Standard Instructor Completion**

In [3]:
import zyx
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

# We can pass in a system prompt to change the behavior of the assistant
response = zyx.completion(
    "Create a mock person",
    response_model = Person
)

# Lets print the full response object
print(response)

print(f"Person Name: '{response.name}', Person Age: '{response.age}'")

Person(name='John Doe', age=30)

Person Name: 'John Doe', Person Age: '30'

### **You also have the ability to change the instructor parsing mode as a parameter**

In [4]:
import zyx
from pydantic import BaseModel

class Person(BaseModel):
    alias : str
    real_name: str
    age: int

# We can pass in a system prompt to change the behavior of the assistant
response = zyx.completion(
    "Who is Spiderman?",
    response_model = Person,
    model = "ollama/llama3.1", # Works with any LiteLLM compatible model
    mode = "md_json" # This is the default mode, but we can also use "json", or "tools"
)

# Lets print the full response object
print(response)

print(f"Alias: '{response.alias}', Real Name: '{response.real_name}', Age: '{response.age}'")

Person(alias='Spiderman', real_name='Peter Parker', age=25)

Alias: 'Spiderman', Real Name: 'Peter Parker', Age: '25'

## **Tool Calling & Execution**

The completion function has built in support for tool calling and execution. </br>

You can pass in a list of tools to the <code>tools</code> parameter. </br>
A tool can be:

- A python function
- An OpenAI formatted tool
- A Pydantic model

> **Note** : zyx also comes with a few prebuilt tools, the web search tool however, requires a Tavily API key.

### **Tool Calling**

Like most LLM frameworks, zyx returns tool call outputs.

In [5]:
# Lets return a tool call
import zyx

# Lets use the prebuilt web search tool!
response = zyx.completion(
    "Who won the 2024 Euro Cup Final?",
    tools = [zyx.tools.web_search],
    run_tools = False # This wont execute the tool, it will just return the tool call
)

print(response.choices[0].message.tool_calls)

[
    ChatCompletionMessageToolCall(
        function=Function(
            arguments='{"query":"2024 Euro Cup Final winner","max_results":"5"}',
            name='web_search'
        ),
        id='call_yKFgoci3OCOXy6zza7vVODzC',
        type='function'
    )
]

### **Tool Execution**

Now lets execute the tool call


In [8]:
# Lets execute a tool call
import zyx

# If Running in Notebook, set the TAVILY_API_KEY environment variable explicitly
import os
os.environ["TAVILY_API_KEY"] = "tvly-xxxx"

# Lets use the prebuilt web search tool!
response = zyx.completion(
    "Who won the 2024 Euro Cup Final?",
    tools = [zyx.tools.web_search],
    run_tools = True # This will execute the tool now
)

print(response.choices[0].message.content)

Spain won the 2024 Euro Cup Final, defeating England 2-1. The decisive goal was scored by substitute Mikel 
Oyarzabal in the 86th minute. This victory marked Spain's fourth European championship title. You can find more 
details about the match (https://en.wikipedia.org/wiki/UEFA_Euro_2024_Final).